<a href="https://colab.research.google.com/github/easyhardhoon/machine_learning_basic/blob/master/ML_RE8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ML16~ML17

**1. conv 구현**

In [ ]:
from util import im2col,col2im
import numpy as np

class Convolution: 
  def __init__(self,W,b,stride=1,pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad
  def forward(self,x):
    FN,C,FH,FW = self.W.shape 
    N,C,H,W = x.shape 
    out_h = int(1+(H+2*self.pad-FH) / self.stride)
    out_w = int(1+(W+2*self.pad-FW) / self.stride)

    col = im2col(x,FH,FW,self.stride,self.pad) 
    col_W = self.W.reshape(FN,-1).T 
    out = np.dot(col,col_W) + self.b 
    out = out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
        return dx
    return out

**2. pooling 구현**

1. 입력 데이터 전개

2. 행별 최댓값 구하기(최대풀링)

3. 적절한 모양으로 다시 reshape

In [ ]:
class Pooling:
  def __init__(self,pool_h,pool_w,stride=1,pad=0):  
    self.pool_h = pool_h
    self,pool_w = pool_w
    self.stride = stride
    self.pad = pad
  def forward(self,x):
    N,C,H,W = x.shape
    out_h = int(1+(H-self.pool_h) / self.stride)
    out_w = int(1+(W-self.pool_w) / self.stride)
    #전개
    col = im2col(x,self.pool_h, self.pool_w, self.stride, self.pad) 
    col = col.reshape(-1,self.pool_h*self.pool_w) #
    #최댓값 구하기
    out = np.max(col,axis=1)
    #성형
    out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)
    return out

  def backward(self, dout): #RELU의 흐름과 유사합니다.
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)

        return dx

**3. CNN 구현**

In [ ]:
import sys, os
sys.path.append(os.pardir)  
import pickle
import numpy as np
from collections import OrderedDict
from layers import *
from gradient import numerical_gradient


class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        acc = 0.0
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)       
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])
        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

**4. CNN 시각화**

학습 전 가중치와 학습 후 가중치의 값을 비교해본다. (ML 16 참고)

최초의 학습되지 않은 가중치는 무작위로 초기화되어 흑백 정도에 규칙성이 없다.

하지만 학습 이후의 가중치(필터)는 규칙성이 있는 이미지가 된다. (흑백 경계선, 덩어리 )

(**이미지에서는 가장 작은 값(0)이 검은색, 가장 큰 값(255)는 흰색으로 정규화된다**)

---

즉, 필터는 경계선과 덩어리를 중심으로 값을 최적화한다. 

예를 들어 세로로 경계선이 있는 필터를 
적용시키면 출력 이미지의 세로 에지가 반응한다. 

이 과정에서 규칙이 없었던 데이터가 규칙이 있는 형태로 바뀐다.  

**이는, 매개변수의 최적화를 통한 학습과 예측의 과정을 시각화 한 것이라고 볼 수 있다.**

합성곱 계층의 필터는 에지나 블롭 등의 원시적인 정보를 추출할 수 있으며, 이러한 정보가 다음 계층에 전달될 수 있다.

**5. 다양한 CNN**

지금까지는 simpleConvNet에 해당. 합성곱 계층을 1개만 사용한 경우였다. 

합성곱 계층이 1개만 사용되면 에지나 블롭 같은 저수준 정보가 추출된다. 

하지만 **CNN을 겹겹이 쌓는다면** ( ex) AlexNet ) 

1번째 층은 에지와 블롭

3번째 층은 텍스쳐

5번째 층은 사물의 일부

마지막 affine 일때에는 사물의 전체 클래스의 정보를 추출할 수 있게 된다.

**각 층의 필터가 학습을 통해 점점 진화하여 최종 예측 모델에서 각 층을 거쳐 고급 정보를 추출할 수 있게 된다.**

---


✅정리


    최초의 CNN에서는 원시 데이터(아무 규칙이 없는)에서 에지와 블롭을 추출하여 1차로 데이터를 가공하고
    그다음 CNN이 겹겹이 이어지면 점점 더 데이터가 가공되어 실제 형태처럼 진화해나간다
    이렇게 CNN을 여러겹 쌓으면, 층이 깊어지면서 더 복잡하고 추상화된 고급 정보가 추출된다.
    다시 말하면, 사물의 의미를 이해하도록 변화한다. 이게 딥러닝의 목표이다.

---

CNN의 종류들에는 LeNet, AlexNet등이 있다.

**사실, 여러 네트워크가 개발이 되었지만 딥러닝의 발전을 이끈 것은 빅데이터와 GPU기술이다.**

⚾빅데이터

    대량의 데이터를 누구나 얻을 수 있다.
    매개변수 fitting을 위한 방대한 시험/훈련 데이터를 쉽게 얻을 수 있다.

⚾GPU

    병렬 계산에 특화되어 대량의 연산을 고속으로 수행할 수 있다.
    딥러닝의 방대한 매개변수 학습을 위한 연산과정을 빠르게 할 수 있다

**6. 딥러닝**

층을 깊게 한 심층 신경망. 지금까지의 신경망을 바탕으로 뒷단에 층을 추가.


---

⏰간단한 복습

✈가중치의 초깃값 설정법 : 0.01(from 정규분포), Xavier 초깃값, He초깃값(RELU전용)

✈가중치의 초깃값 설정안하는 고급 기술 : 배치 정규화

✈오버피팅 방지법 : 가중치 감소, 드롭아웃

✈가중치 매개변수 갱신법 : SGD, 모멘텀, AdaGrad, Adam

✈가중치(parameter)가 들어가는 계층 : 합성곱 계층(conv), 완전연결 계층(affine)

---

지금까지의 모든 기술을 동원하여 심층 신경망을 구현하고 정확도를 구해보면

99.38%로 이전의 단순 신경망(98%)보다 상승한 모습을 볼 수 있다.

**즉, 심층 CNN은 정확도가 높고, 잘못 인식한 이미지들도 인간과 비슷한 인식 오류를 저지르기 때문에, 잠재력이 크다는 것을 알 수 있다.**

**7. 기타**

**층을 깊게 하는 이유**

✔신경망의 매개변수 수 감소(7^7 합성곱 1회를 3^3 합성곱 3회로 대체 가능. 궁극적으로 신경망의 표현력 개선)

✔학습의 효율성 증가 (층마다 학습 할 문제 분업)

---

**딥러닝 고속화**

요즘에는 다양한 심층 신경망이 사용되고 있고, 이에 따른 대량의 연산을 위해 복수의 GPU와 여러 기기로 분산 수행(학습)을 하고 있다. ( + 비트 줄이기 기술)

( 분산 학습으로 GPU 100개를 사용하면 40일 걸리는것을 3시간만에 끝낼 수 있다.)

---

**딥러닝의 활용과 미래**

컴퓨터 비젼

    사물 검출, 분할, 사진 캡션 생성 등등

NLP

    RNN(순환 신경망) 사용. 이전에 생성한 정보에 영향을 받는다.

이미지 생성, 스타일 변환

    딥러닝을 활용하여 화가처럼 그림을 그리기. 
    ex) 일반 사진에 고흐의 화풍을 입히기

    입력 이미지 없이 새로운 이미지 그려내기. 
    ex) 아직 아무도 본 적 없는 침실 이미지 생성

자율주행

    분할 기술(컴뷰터 비전) + 딥러닝 인식 기술(SegNet) + ... 

**강화학습**
    
    에이전트는 환경에 맞게 행동을 선택하고, 그 행동에 의해서 환경이 변한다.
    환경이 변하면 어떠한 보상을 얻는다. 더 나은 보상을 받는 쪽으로 에이전트의 행동 지침을 바로잡는다
    즉, 에이전트는 더 좋은 보상을 받기 위해 스스로 학습한다.
    여기서 보상은, 명확한 지표로부터 역산하여 정해야만 한다. 정해진 보상이 아니라 예상 보상이다.

    이러한 강화학습을, 딥러닝을 이용해 구현할 수 있다. (DQN)
    최적 행동 가치 함수를 딥러닝으로 비슷하게 흉내내는 식이다.
    DQN 연구로, 비디오 게임을 자율 학습시켜 사람을 뛰어넘는 수준의 조작을 실현하였다. 
    여기서 핵심은, DQN에게 주어지는 입력 데이터는 오직 비디오 게임의 영상뿐이라는 것이다.
    이 점으로 인해, DQN의 응용 가능성은 매우 높다

    알파고에도 딥러닝과 강화학습이 이용되었다.
    알파고는 3만개의 프로 기보를 보며 학습한 후 알파고 자신과 스스로 맞붙는 대결을 반복하며 수련하였다.
    




✅참고로

"이미지 데이터"와 "정답 레이블"을 짝지은 데이터셋(ex MNIST)을 이용하는 학습을 **지도학습**이라 하고

단지 대량의 이미지 데어터만 주어진 데이터셋(ex DCGAN)을 이용하는 학습을 **자율학습(비지도학습)**이라고 한다

✅즉

⚡지도학습 : 정답을 알려주며 학습. (분류, 회귀)

⚡비지도학습 : 정답을 알려주지 않음. 비슷한 데이터들 군집화. 

⚡강화학습 : 분류할 데이터가 존재하지 않음. 데이터가 있어도 정답이 정해지지 않음.
자신이 한 행동에 대해 보상을 받으며 학습


#마무리하며

이렇게 [밑바닥부터 시작하는 딥러닝 1]의 복습을 완료하였습니다. [22.02.09 ~ 22.02.20 ] 